In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
from scipy import signal
import matplotlib.pylab as pl

In [2]:
def costMatrixConvolution(b, nl, nc, F, eps, alpha):    
    # Construction du vecteur de distance sur l'axe des colonnes
    tc = np.linspace(0,1,nc)
    vect1 = np.array(list(reversed(tc)))
    vect2 = np.array(list(tc))
    vect2 = vect2[1:nc]
    G1 = np.exp(-vect1**2/eps)
    G2 = np.exp(-vect2**2/eps)
    G = np.concatenate((G1, G2))
    
    # Construction du vecteur de distance sur l'axe des lignes
    tl = np.linspace(0,1,nl)
    vect1 = np.array(list(reversed(tl)))
    vect2 = np.array(list(tl))
    vect2 = vect2[1:nl]
    V_p1 = np.exp(-vect1**2/eps)
    V_p2 = np.exp(-vect2**2/eps)
    V_p = np.concatenate((V_p1, V_p2))
    
    # Construction du vecteur de distance sur l'axe des couleurs
    tb = np.linspace(0,1,b)
    vect1 = np.array(list(reversed(tb)))
    vect2 = np.array(list(tb))
    vect2 = vect2[1:b]
    V1 = np.exp(-(alpha*vect1**2)/eps)
    V2 = np.exp(-(alpha*vect2**2)/eps)
    V = np.concatenate((V1, V2))
    
    #On init le vecteur R qui va contenir les convolutions K_i*F_i
    R = []
    #On commence à boucler sur le nombre de couleurs
    for i in range(b):
        #On extrait le vecteur Fi de F
        F_i = F[i*(nc*nl):(i+1)*(nc*nl)]

        #On init le vecteur R' qui va contenir les convolutions K_i_j*F_ij
        R_p = []
        
        #On boucle ensuite sur le nombre de lignes pour faire les opérations de convolutions
        for j in range(nl):
            #On extrait le vecteur Fi,j de F pour faire les convolutions dessus
            F_i_j = F_i[j*(nc):(j+1)*nc]
            #Convolution sur ce vecteur
            tmp = sc.signal.fftconvolve(G, F_i_j)
            Res = tmp[nc-1:2*nc - 1]
            R_p.append(Res)
            
        R_p = np.array(R_p) #Met en array pour pouvoir faire les calculs dessus -> On a ici le vecteur R'(0->nl-1)
                            #contenant les convolutions : les vecteurs R'(i) sont de taille nc pour 0<i<nl-1

        #On construit ici la convolution : V'*R'
        tmp = np.array(sc.signal.fftconvolve(V_p, R_p[:,0]))
        Qi = tmp[nl - 1:2*nl - 1]
        for k in range(nc-1):
            tmp = sc.signal.fftconvolve(V_p, R_p[:,k+1])
            Qi = np.block([Qi, tmp[nl - 1:2*nl - 1]])  
            
        R.append(Qi)
    
    R = np.array(R) #Met en array pour pouvoir faire les calculs dessus -> On a ici le vecteur R(0->b-1)
                            #contenant les convolutions : les vecteurs R(i) sont de taille nl pour 0<i<b-1
    print(R)
    print('\n')
    #On construit ici la convolution : V*R qui est le résultat final K*F
    tmp = np.array(sc.signal.fftconvolve(V, R[:,0]))
    print(R[:,0])
    Q = tmp[b - 1:2*b - 1]
    for l in range((nl*nc)-1):
        tmp = sc.signal.fftconvolve(V, R[:,l+1])
        Q = np.block([Q, tmp[b - 1:2*b - 1]])
        
    return Q

In [3]:
def costMatrixConstruction(b, nl, nc, F, eps, alpha):#Cette fonction nécessite que b et nl soient puissances de 2
    tc = np.linspace(0,1,nc)
    [Y,X] = np.meshgrid(tc,tc)
    Kc = np.exp(-(X-Y)**2/eps)
    
    Kl = np.block([[Kc, Kc],[Kc, Kc]])
    for i in range(nl-1):
        while(Kl.shape[0]<nl*nc):
            Kl = np.block([[Kl, Kl], [Kl, Kl]])

    tl = np.linspace(0,1,nl)
    for i in range(nl*nc):
        for j in range(nl*nc):
            if (i//(nc)) -(j//(nc)) != 0 :
                Kl[i,j] = Kl[i,j]*np.exp(-((tl[i//(nc)]-tl[j//(nc)])**2)/eps)


    Kb = np.block([[Kl,Kl],[Kl, Kl]])
    for i in range(b-1):
        while(Kb.shape[0]<nl*nc*b):
            Kb = np.block([[Kb, Kb], [Kb, Kb]])

    print(Kb.shape)

    tb = np.linspace(0,1,b)

    for i in range(b*nl*nc):
        for j in range(b*nl*nc):
            if (i//(nc*nl)) -(j//(nc*nl)) != 0 :
                Kb[i,j] = Kb[i,j]*np.exp(-alpha*((tb[i//(nc*nl)]-tb[j//(nc*nl)])**2)/eps)


    return Kb

In [227]:
def costMatrixConvolution2(b, nl, nc, F, eps, alpha):    
    # Construction du vecteur de distance sur l'axe des colonnes
    tc = np.linspace(0,1,nc)
    vect1 = np.array(list(reversed(tc)))
    vect2 = np.array(list(tc))
    vect2 = vect2[1:nc]
    G1 = np.exp(-vect1**2/eps)
    G2 = np.exp(-vect2**2/eps)
    G = np.concatenate((G1, G2))

    # Construction du vecteur de distance sur l'axe des lignes
    tl = np.linspace(0,1,nl)
    vect1 = np.array(list(reversed(tl)))
    vect2 = np.array(list(tl))
    vect2 = vect2[1:nl]
    V_p1 = np.exp(-vect1**2/eps)
    V_p2 = np.exp(-vect2**2/eps)
    V_p = np.concatenate((V_p1, V_p2))
    
    # Construction du vecteur de distance sur l'axe des couleurs
    tb = np.linspace(0,1,b)
    vect1 = np.array(list(reversed(tb)))
    vect2 = np.array(list(tb))
    vect2 = vect2[1:b]
    V1 = np.exp(-(alpha*vect1**2)/eps)
    V2 = np.exp(-(alpha*vect2**2)/eps)
    V = np.concatenate((V1, V2))
    
    
    #On init le vecteur R qui va contenir les convolutions K_i*F_i
    R = []
    #On commence à boucler sur le nombre de couleurs
    for i in range(b):
        #On extrait le vecteur Fi de F
        F_i = F[i*(nc*nl):(i+1)*(nc*nl)]
        #On init le vecteur R' qui va contenir les convolutions K_i_j*F_ij
        R_p = []
        
        #On boucle ensuite sur le nombre de lignes pour faire les opérations de convolutions
        for j in range(nl):
            #On extrait le vecteur Fi,j de F pour faire les convolutions dessus
            F_i_j = F_i[j*(nc):(j+1)*nc]
            #Convolution sur ce vecteur
            tmp = sc.signal.fftconvolve(G, F_i_j)
            Res = tmp[nc-1:2*nc - 1]
            R_p.append(Res)
            
        R_p = np.array(R_p) #Met en array pour pouvoir faire les calculs dessus -> On a ici le vecteur R'(0->nl-1)
                            #contenant les convolutions : les vecteurs R'(i) sont de taille nc pour 0<i<nl-1
        #print(R_p)
        
        #On construit ici la convolution : V'*R'
        tmp = np.array(sc.signal.fftconvolve(V_p, R_p[:,0]))
        Qi = tmp[nl - 1:2*nl - 1]
        for k in range(nc-1):
            tmp = sc.signal.fftconvolve(V_p, R_p[:,k+1])
            Qi = np.block([[Qi], [tmp[nl - 1:2*nl - 1]]])

        Qi = np.transpose(Qi)
        A = Qi[0]
        for k in range(Qi.shape[0]-1):
            A = np.concatenate((A, Qi[k+1]), axis=None)
        
        R.append(A)

    R = np.array(R) #Met en array pour pouvoir faire les calculs dessus -> On a ici le vecteur R(0->b-1)
                       #contenant les convolutions : les vecteurs R(i) sont de taille nl pour 0<i<b-1
    #print('\n')
    #print('R is :', R)
    #print('\n')
    #On construit ici la convolution : V*R qui est le résultat final K*F

    tmp = np.array(sc.signal.fftconvolve(V, R[:,0]))
    Q = tmp[b - 1:2*b - 1]
    for l in range((nl*nc)-1):
        tmp = sc.signal.fftconvolve(V, R[:,l+1])
        Q = np.block([[Q], [tmp[b - 1:2*b - 1]]])
    
    
    Q = np.transpose(Q)
    finalRes = Q[0]
    for k in range(Q.shape[0]-1):
            finalRes = np.concatenate((finalRes, Q[k+1]), axis=None)
    #print('\n')
    #print('resultat final', finalRes)
    
    return finalRes

In [231]:
b = 2
nl = 4
nc = 4
eps = 0.01
alpha = 2
F = np.arange(b*nc*nl)
K = costMatrixConstruction(b, nl, nc, F, eps, alpha)
Qnaif = np.dot(K, F)
print(Qnaif)
print('\n')
Qconv = costMatrixConvolution2(b, nl, nc, F, eps, alpha)
print(Qconv)

(32, 32)
[7.47278094e-05 1.00010462e+00 2.00014946e+00 3.00013451e+00
 4.00019429e+00 5.00029891e+00 6.00035869e+00 7.00029891e+00
 8.00037364e+00 9.00053804e+00 1.00005978e+01 1.10004783e+01
 1.20003139e+01 1.30005231e+01 1.40005679e+01 1.50003736e+01
 1.60005530e+01 1.70008220e+01 1.80008668e+01 1.90006128e+01
 2.00009117e+01 2.10012554e+01 2.20013152e+01 2.30010163e+01
 2.40010910e+01 2.50014946e+01 2.60015543e+01 2.70011956e+01
 2.80007921e+01 2.90012405e+01 3.00012853e+01 3.10008519e+01]


[7.47278094e-05 1.00010462e+00 2.00014946e+00 3.00013451e+00
 4.00019429e+00 5.00029891e+00 6.00035869e+00 7.00029891e+00
 8.00037364e+00 9.00053804e+00 1.00005978e+01 1.10004783e+01
 1.20003139e+01 1.30005231e+01 1.40005679e+01 1.50003736e+01
 1.60005530e+01 1.70008220e+01 1.80008668e+01 1.90006128e+01
 2.00009117e+01 2.10012554e+01 2.20013152e+01 2.30010163e+01
 2.40010910e+01 2.50014946e+01 2.60015543e+01 2.70011956e+01
 2.80007921e+01 2.90012405e+01 3.00012853e+01 3.10008519e+01]


In [233]:
b = 4
nl = 8
nc = 12
eps = 0.01
alpha = 1
F = np.ones(b*nc*nl)

In [234]:
K = costMatrixConstruction(b, nl, nc, F, eps, alpha)
Qnaif = np.dot(K, F)
print(Qnaif)
print('\n')
Qconv = costMatrixConvolution2(b, nl, nc, F, eps, alpha)
print(Qconv)

(384, 384)
[1.66692543 2.1615135  2.20295929 2.20362437 2.20362642 2.20362642
 2.20362642 2.20362642 2.20362437 2.20295929 2.1615135  1.66692543
 1.85854625 2.40998952 2.4561997  2.45694123 2.45694351 2.45694351
 2.45694351 2.45694351 2.45694123 2.4561997  2.40998952 1.85854625
 1.85896649 2.41053444 2.45675507 2.45749678 2.45749905 2.45749906
 2.45749906 2.45749905 2.45749678 2.45675507 2.41053444 1.85896649
 1.85896651 2.41053446 2.45675509 2.4574968  2.45749908 2.45749908
 2.45749908 2.45749908 2.4574968  2.45675509 2.41053446 1.85896651
 1.85896651 2.41053446 2.45675509 2.4574968  2.45749908 2.45749908
 2.45749908 2.45749908 2.4574968  2.45675509 2.41053446 1.85896651
 1.85896649 2.41053444 2.45675507 2.45749678 2.45749905 2.45749906
 2.45749906 2.45749905 2.45749678 2.45675507 2.41053444 1.85896649
 1.85854625 2.40998952 2.4561997  2.45694123 2.45694351 2.45694351
 2.45694351 2.45694351 2.45694123 2.4561997  2.40998952 1.85854625
 1.66692543 2.1615135  2.20295929 2.20362437 2.2036